## Frequency Analysis

In [10]:
%env ASTERIA=/home/jakob/software/ASTERIA/ASTERIA
from asteria.simulation import Simulation
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.fft import fft, ifft, fftfreq
from scipy.stats import skewnorm, norm
from scipy.signal import fftconvolve as convolve

env: ASTERIA=/home/jakob/software/ASTERIA/ASTERIA


In [2]:
def moving_average(a, n=3, zero_padding = False, const_padding = False):
    if zero_padding:
        a = np.insert(a, np.zeros(n-1,dtype=int), np.zeros(n-1), axis=-1)
        a = np.roll(a, -int((n-1)/2), axis=-1)
    if const_padding:
        l1 = int(n/2)
        if n%2 != 1: # n is even
            l2 = l1-1
        else: # n is odd
            l2 = l1
            ind2 = -np.arange(1,(n+1)/2).astype(int)
        a = np.insert(a, np.zeros(l1, dtype=int), np.ones(l1)*a[0])
        a = np.insert(a, -np.ones(l2, dtype=int), np.ones(l2)*a[-1])
    ret = np.cumsum(a, dtype=float, axis=-1)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [3]:
#Returns binning needed to resolve a frequency.
def get_binning(sim, dt, frequency=80*u.Hz): 
    duration = sim.time[-1]-sim.time[0]
    samples = (duration/dt.to(u.s)).value
    return int((1/frequency*samples/duration).value) #binning needed to filter out sinals with f>f_lb_sasi

In [4]:
# Binning of the moving average filter defined by the Nyquist frequency of the SASI modulation
# If the sampling was to be finer the binning could be done finer, however this comes at the expense of lower
# statistics. Therefore we keep the sampling rate at 1 ms.

def get_average_signal(sim, dt, distance, frequency=80*u.Hz):
    binning = get_binning(sim, dt, frequency)

    #detector_signal s0 is not drawn from distribution
    t, s0_i3 = sim.detector_signal(dt=dt, subdetector='i3')
    t, s0_dc = sim.detector_signal(dt=dt, subdetector='dc')
    t, s0_md = sim.detector_signal(dt=dt, subdetector='md')

    s0_ic86 = s0_i3 + s0_dc
    s0_gen2 = s0_i3 + s0_dc + s0_md
    t = t.to(u.ms)

    #averaged signal sa binning size defined by the Nyquist frequency of the SASI modulation and constant padding
    sa_ic86 = moving_average(s0_ic86, n=binning, const_padding=True)
    sa_gen2 = moving_average(s0_gen2, n=binning, const_padding=True)
    
    return sa_gen2, sa_ic86

In [5]:
def get_template(frequency, amplitude, n_period, binning):
    frequency, binning = frequency.value, binning.to(u.s).value
    duration = 1/frequency * n_period
    x = np.arange(0,duration+binning,binning)
    template = np.sin(frequency*x*2*np.pi) * amplitude
    return x, template

In [6]:
# The main script that performs the analysis
def signal_processing(sim, dt, distance, trials, dist=skewnorm,
                     temp_freq = 80*u.Hz, temp_periods = 8, temp_amplitude = 1,
                     return_raw=False,
                     return_percentage=False,
                     return_correlation=False,
                     return_max_correlation=False,
                     return_test_statistics=False,
                     return_significance=False):

         
    # scale simulation with distance
    sim.scale_result(distance=distance)
    
    sa_gen2, sa_ic86 = get_average_signal(sim, dt=dt, distance=distance)
    
    # time and signal realization for each detector component
    t, s_i3 = sim.detector_hits(dt=dt, subdetector='i3', size=trials)
    t, s_dc = sim.detector_hits(dt=dt, subdetector='dc', size=trials)
    t, s_md = sim.detector_hits(dt=dt, subdetector='md', size=trials)
    #t, s_ws = sim.detector_hits(dt=dt, subdetector='ws')
    t = t.to(u.ms)

    # random background realization
    b_i3 = sim.detector.i3_bg(dt=dt, size=s_i3.shape[1]*trials)
    b_dc = sim.detector.dc_bg(dt=dt, size=s_dc.shape[1]*trials)
    b_md = sim.detector.md_bg(dt=dt, size=s_md.shape[1]*trials)
    #b_ws = sim.detector.ws_bg(dt=dt, size=len(s_ws)*trials)
    
    # reshape background
    b_i3 = b_i3.reshape(trials,s_i3.shape[1])
    b_dc = b_dc.reshape(trials,s_dc.shape[1])
    b_md = b_md.reshape(trials,s_md.shape[1])
    
    # analysis window mask
    t_mask = np.logical_and(t>=time_wind[0], t<=time_wind[1])    
    
    # generate template of same shape in dimension 0 as data to convolve with
    t_template, template = get_template(frequency=temp_freq, amplitude = temp_amplitude, n_period=temp_periods, binning=dt)
    template = np.tile(template, trials)
    template = template.reshape(trials,-1)
    
    ### SIGNAL TRIALS ###
    s_ic86 = s_i3 + s_dc
    s_gen2 = s_i3 + s_dc + s_md
    b_ic86 = b_i3 + b_dc
    b_gen2 = b_i3 + b_dc + b_md

    # combined signal S: signal + background
    S_ic86 = s_ic86 + b_ic86
    S_gen2 = s_gen2 + b_gen2

    # background subtraction Sb: signal + background - average background
    Sb_ic86 = S_ic86 - ba_ic86
    Sb_gen2 = S_gen2 - ba_gen2

    # percentage deviation Sp: (average signal + background - average background)/average signal
    Sp_ic86 = (Sb_ic86/sa_ic86)-1
    Sp_gen2 = (Sb_gen2/sa_gen2)-1
    
    # cross correlation in analysis window
    Sc_ic86 = -convolve(Sp_ic86[:,t_mask], template, mode='same', axes=-1)
    Sc_gen2 = -convolve(Sp_gen2[:,t_mask], template, mode='same', axes=-1)

    # maximum cross correlation
    Sm_ic86 = np.nanmax(Sc_ic86, axis=-1)
    Sm_gen2 = np.nanmax(Sc_gen2, axis=-1)
    
    ### BACKGROUND TRIALS ###
    # combined averaged signal B: average signal + background
    B_ic86 = sa_ic86 + b_ic86
    B_gen2 = sa_gen2 + b_gen2

    # background subtraction Bb: average signal + background - average background
    Bb_ic86 = B_ic86 - ba_ic86
    Bb_gen2 = B_gen2 - ba_gen2

    # deviation Bd: average signal + background - average background - average signal = background - average background
    Bd_ic86 = b_ic86 - ba_ic86
    Bd_gen2 = b_gen2 - ba_gen2

    # percentage deviation Bp: (average signal + background - average background)/average signal
    Bp_ic86 = (Bb_ic86/sa_ic86)-1
    Bp_gen2 = (Bb_gen2/sa_gen2)-1
    
    # cross correlation in analysis window
    Bc_ic86 = -convolve(Bp_ic86[:,t_mask], template, mode='same', axes=-1)
    Bc_gen2 = -convolve(Bp_gen2[:,t_mask], template, mode='same', axes=-1)

    # maximum cross correlation
    Bm_ic86 = np.nanmax(Bc_ic86, axis=-1)
    Bm_gen2 = np.nanmax(Bc_gen2, axis=-1)
    
    # background fit parameters
    Bfp_ic86 = dist.fit(Bm_ic86)
    Bfp_gen2 = dist.fit(Bm_gen2)

    # fitted background distribution
    Bf_ic86 = dist(*Bfp_ic86)
    Bf_gen2 = dist(*Bfp_gen2)
    
    # median, 16% and 84% quantiles of TS distribution
    Sts_gen2 = np.array([np.median(Sm_gen2), np.quantile(Sm_gen2, 0.16), np.quantile(Sm_gen2, 0.84)])
    Sts_ic86 = np.array([np.median(Sm_ic86), np.quantile(Sm_ic86, 0.16), np.quantile(Sm_ic86, 0.84)])
    Bts_gen2 = np.array([np.median(Bm_gen2), np.quantile(Bm_gen2, 0.16), np.quantile(Bm_gen2, 0.84)])
    Bts_ic86 = np.array([np.median(Bm_ic86), np.quantile(Bm_ic86, 0.16), np.quantile(Bm_ic86, 0.84)])

    ### SIGNIFICANCE ###
    # take the median and quantiles of the signal trials and compute the corresponding p-value/significance 
    # from the bkg distribution
    Z_ic86 = []
    Z_gen2 = []
    for i in range(3):  # loop over Sts values (median, 16%, 84% quantiles)   
    # p-value of signal given a background distribution

        p_ic86 = Bf_ic86.sf(Sts_ic86[i])
        p_gen2 = Bf_gen2.sf(Sts_gen2[i])

        # two-sided Z score corresponding to the respective p-value, survival probability = 1 - cdf
        z_ic86 = norm.isf(p_ic86/2)
        z_gen2 = norm.isf(p_gen2/2)

        Z_ic86.append(z_ic86)
        Z_gen2.append(z_gen2)
    
    if return_raw:
        return t, s_gen2, sa_gen2, b_gen2, s_ic86, sa_ic86, b_ic86
    
    if return_percentage:
        return t, Sp_gen2, Sp_ic86, Bp_gen2, Bp_ic86
    
    if return_correlation:
        return t[t_mask], Sc_gen2, Sc_ic86, Bc_gen2, Bc_ic86
    
    if return_max_correlation:
        return t, Sm_gen2, Sm_ic86, Bm_gen2, Bm_ic86, Bf_gen2, Bf_ic86
        
    if return_test_statistics:
        return Sts_gen2, Sts_ic86, Bts_gen2, Bts_ic86
    
    if return_significance:
        return Sts_gen2, Sts_ic86, Bts_gen2, Bts_ic86, Z_gen2, Z_ic86

In [7]:
def get_sim(index):
    dt = 1 * u.ms

    #default settings
    include_wls = False #"False", "True"

    # template parameters
    temp_freq = 80 * u.Hz
    temp_periods = 8
    temp_amplitude = 1
    
    stime = "wide" # "wide", "narrow"
    sfreq = 0 * u.Hz # freq. modulation: 0 Hz, +- 10 Hz

    smix = "NoTransformation" # "NoTransformation", "CompleteExchange", "AdiabaticMSW"
    shier = "normal" # "normal", "inverted"
    smodel = "Tamborra_2014_20M" # "Tamborra_2014_20M", "Tamborra_2014_27M"
    sdir = 1

    if index == 1:
        include_wls = True

    elif index == 2:
        stime = "narrow"

    elif index == 3:
        stime = "narrow"
        include_wls = True

    elif index == 4:
        sfreq = 10 * u.Hz

    elif index == 5:
        sfreq = 10 * u.Hz
        include_wls = True

    elif index == 6:
        sfreq = -10 * u.Hz

    elif index == 7:
        sfreq = -10 * u.Hz
        include_wls = True
        
    elif index == 8:
        temp_periods *= 1/2

    elif index == 9:
        temp_periods *= 1/2
        include_wls = True

    elif index == 10:
        temp_periods *= 2

    elif index == 11:
        temp_periods *= 2
        include_wls = True
        
    elif index == 12:
        smix = "CompleteExchange"

    elif index == 13:
        smix = "CompleteExchange"
        include_wls = True  

    elif index == 14:
        smix = "AdiabaticMSW"

    elif index == 15:
        smix = "AdiabaticMSW"
        include_wls = True  

    elif index == 16:
        smix = "AdiabaticMSW"
        shier = "inverted"

    elif index == 17:
        smix = "AdiabaticMSW"
        shier = "inverted"
        include_wls = True

    elif index == 18:
        smodel = "Tamborra_2014_27M"
        sdir = 1 

    elif index == 19:
        smodel = "Tamborra_2014_27M"
        sdir = 1  
        include_wls = True

    elif index == 20:
        smodel = "Tamborra_2014_27M"
        sdir = 2 

    elif index == 21:
        smodel = "Tamborra_2014_27M"
        sdir = 2  
        include_wls = True

    elif index == 22:
        smodel = "Tamborra_2014_27M"
        sdir = 3 

    elif index == 23:
        smodel = "Tamborra_2014_27M"
        sdir = 3  
        include_wls = True

    if smodel == "Tamborra_2014_11M":
        #Does not show SASI, maybe LESA
        Emin, Emax, dE = 0*u.MeV, 100*u.MeV, 1*u.MeV
        tmin, tmax, dt = 0.010*u.s, 0.34979*u.s, dt
        mass = 11.2*u.Msun
        sdir = 1

    elif smodel == "Tamborra_2014_20M":
        Emin, Emax, dE = 0*u.MeV, 100*u.MeV, 1*u.MeV
        tmin, tmax, dt = 0.006*u.s, 0.338*u.s, dt
        mass = 20*u.Msun
        sdir = 1

    elif smodel == "Tamborra_2014_27M":
        Emin, Emax, dE = 0*u.MeV, 100*u.MeV, 1*u.MeV
        tmin, tmax, dt = 0.011*u.s, 0.552*u.s, dt
        mass = 27*u.Msun

    else:
        raise ValueError('Model '+smodel+' does not exist!')
    
    if smodel == "Tamborra_2014_20M":
        if stime == "wide":
            time_wind = [0.075, 0.338] * u.s

        elif stime == "narrow":
            time_wind = [0.150, 0.300] * u.s
    
    if smodel == "Tamborra_2014_27M":
        time_wind = [0.075, 0.300] * u.s

    
    temp_freq = 80 * u.Hz + sfreq  

    print(smodel, sdir)
    print(smix)
    print(shier)
    print(time_wind)
    print('Template: {} Hz, {} periods, {}x scaling'.format(temp_freq, temp_periods, temp_amplitude))
    print('WLS ?: {}'.format(include_wls))
    
    model = {'name': 'Tamborra_2014',
                 'param':{
                 'progenitor_mass': mass,
                 'direction': sdir}
                }

    sim = Simulation(model=model,
                     distance=10* u.kpc, 
                     Emin=Emin, Emax=Emax, dE=dE,
                     tmin=tmin, tmax=tmax, dt=dt,
                     hierarchy = shier,
                     mixing_scheme = smix,
                     geomscope = 'Gen2',
                     include_wls = include_wls)
    sim.run()
    return sim, time_wind, temp_freq, temp_periods, temp_amplitude, smodel, sdir, smix, shier, stime

In [14]:
for i in range(24):
    dt = 1 * u.ms

    sim, time_wind, temp_freq, temp_periods, temp_amplitude, smodel, sdir, smix, shier, stime = get_sim(i)

    #average background given by the mean of the sensor distribution and scaled to the full detector
    ba_ic86 = (sim.detector.n_i3_doms*sim.detector.i3_dom_bg_mu + sim.detector.n_dc_doms*sim.detector.dc_dom_bg_mu)*dt/(1*u.s)
    ba_gen2 = (sim.detector.n_i3_doms*sim.detector.i3_dom_bg_mu + sim.detector.n_dc_doms*sim.detector.dc_dom_bg_mu + sim.detector.n_md*sim.detector.md_bg_mu)*dt/(1*u.s)

    distances = np.arange(1,41,1) * u.kpc
    trials = 10000
    freq_sasi = 80 * u.Hz

    
    Sts_gen2, Sts_ic86, Bts_gen2, Bts_ic86, Z_gen2, Z_ic86 = [], [], [], [], [], []
    for d in distances:
        print('distance {:.0f}'.format(d))
        sts_gen2, sts_ic86, bts_gen2, bts_ic86, z_gen2, z_ic86 = signal_processing(sim, dt=dt, distance=d, trials=trials, 
        temp_freq = temp_freq, temp_periods = temp_periods, temp_amplitude = temp_amplitude, return_significance=True)

        Sts_gen2.append(sts_gen2)
        Sts_ic86.append(sts_ic86)
        Bts_gen2.append(bts_gen2)
        Bts_ic86.append(bts_ic86)
        Z_gen2.append(z_gen2)
        Z_ic86.append(z_ic86)

    Sts_gen2 = np.array(Sts_gen2)
    Sts_ic86 = np.array(Sts_ic86)
    Bts_gen2 = np.array(Bts_gen2)
    Bts_ic86 = np.array(Bts_ic86)
    Z_gen2 = np.array(Z_gen2)
    Z_ic86 = np.array(Z_ic86)
    
    data = [distances, Sts_gen2, Sts_ic86, Bts_gen2, Bts_ic86, Z_gen2, Z_ic86]
    if i%2==1:
        filename = './files/temp_'+smodel+'_dir='+str(sdir)+'_mix='+smix+'_hierarchy='+shier+'_time'+stime+'_template_{:.0f}Hz_{:.0f}periods_{}scaling'.format(temp_freq.value, temp_periods, temp_amplitude)+'_IC86+Gen2+WLS.pkl'
    else:
        filename = './files/temp_'+smodel+'_dir='+str(sdir)+'_mix='+smix+'_hierarchy='+shier+'_time'+stime+'_template_{:.0f}Hz_{:.0f}periods_{}scaling'.format(temp_freq.value, temp_periods, temp_amplitude)+'_IC86+Gen2.pkl'

    file = open(filename, 'wb')
    pickle.dump(data,file)

Tamborra_2014_27M 2
NoTransformation
normal
[0.075 0.3  ] s
Template: 80.0 Hz Hz, 8 periods, 1x scaling
WLS ?: False
distance 1 kpc
distance 2 kpc
distance 3 kpc
distance 4 kpc
distance 5 kpc
distance 6 kpc
distance 7 kpc
distance 8 kpc
distance 9 kpc
distance 10 kpc
distance 11 kpc
distance 12 kpc
distance 13 kpc
distance 14 kpc
distance 15 kpc
distance 16 kpc
distance 17 kpc
distance 18 kpc
distance 19 kpc
distance 20 kpc
distance 21 kpc
distance 22 kpc
distance 23 kpc
distance 24 kpc
distance 25 kpc
distance 26 kpc
distance 27 kpc
distance 28 kpc
distance 29 kpc
distance 30 kpc
distance 31 kpc
distance 32 kpc
distance 33 kpc
distance 34 kpc
distance 35 kpc
distance 36 kpc
distance 37 kpc
distance 38 kpc
distance 39 kpc
distance 40 kpc
Tamborra_2014_27M 2
NoTransformation
normal
[0.075 0.3  ] s
Template: 80.0 Hz Hz, 8 periods, 1x scaling
WLS ?: True
distance 1 kpc
distance 2 kpc
distance 3 kpc
distance 4 kpc
distance 5 kpc
distance 6 kpc
distance 7 kpc
distance 8 kpc
distance 9 kpc
d